In [206]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import category_encoders as ce 

import xgboost as xgb
pd.set_option('future.no_silent_downcasting', True)
df = pd.read_csv('train_heart.csv', sep=',')


In [207]:
features_to_be_scaled = [feature for feature in df.columns if len(df[feature].unique()) < 15]
features_to_be_scaled.remove('HeartDisease')
print(features_to_be_scaled)

temp_df = pd.get_dummies(df, columns=features_to_be_scaled)
std = StandardScaler()
temp_df[['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']] = std.fit_transform(temp_df[['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']])
temp_df

['Sex', 'ChestPainType', 'FastingBS', 'RestingECG', 'ExerciseAngina', 'ST_Slope']


,id,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,HeartDisease,Sex_F,Sex_M,ChestPainType_ASY,...,FastingBS_0,FastingBS_1,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,563,0.138742,0.143405,0.071328,-0.400837,0.174902,1,False,True,True,...,False,True,False,False,True,False,True,False,True,False
1,884,1.426535,1.543437,0.832061,-1.116214,0.556505,1,False,True,True,...,True,False,True,False,False,False,True,False,True,False
2,352,0.246058,-0.696614,-1.821229,-1.434160,-1.828517,1,False,True,True,...,True,False,False,False,True,False,True,True,False,False
3,694,0.246058,-0.696614,0.368199,1.665809,-0.111301,0,False,True,False,...,True,False,False,True,False,True,False,False,False,True
4,491,2.285063,2.103449,0.062051,-1.116214,-0.874508,1,False,True,True,...,False,True,False,False,True,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,191,-0.397838,2.103449,0.117714,-0.798269,-0.874508,0,False,True,False,...,True,False,False,False,True,True,False,False,False,True
638,311,0.675322,-0.416608,-1.821229,-1.036728,-0.779107,1,False,True,True,...,False,True,False,True,False,True,False,False,False,True
639,211,-0.397838,0.423411,0.850616,0.155568,-0.874508,1,True,False,False,...,True,False,False,True,False,False,True,False,True,False
640,784,1.211903,0.311409,0.794952,1.506836,0.461104,1,False,True,False,...,False,True,True,False,False,True,False,False,True,False


In [208]:
X = temp_df.drop(['id', 'HeartDisease'], axis=1)
y = temp_df['HeartDisease']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train

,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,...,FastingBS_0,FastingBS_1,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
265,-0.075890,-0.136601,0.006387,0.632486,0.270302,False,True,False,False,True,...,False,True,True,False,False,True,False,True,False,False
573,0.675322,0.983424,0.572299,0.831202,1.605915,True,False,True,False,False,...,True,False,True,False,False,True,False,False,True,False
638,0.675322,-0.416608,-1.821229,-1.036728,-0.779107,False,True,True,False,False,...,False,True,False,True,False,True,False,False,False,True
24,-0.183206,-0.248604,0.080605,1.904268,-0.874508,False,True,False,True,False,...,False,True,False,True,False,True,False,False,False,True
227,0.889954,1.543437,-1.821229,-2.546969,-0.874508,False,True,False,False,True,...,True,False,False,True,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0.997271,-0.024599,-1.821229,-0.241864,-0.779107,True,False,False,True,False,...,True,False,False,True,False,True,False,False,False,True
106,-1.578315,-2.152647,0.024942,1.705552,-0.874508,True,False,False,False,True,...,True,False,False,True,False,True,False,False,False,True
270,-0.505154,0.423411,-0.104940,-0.241864,-0.874508,False,True,True,False,False,...,True,False,False,True,False,True,False,False,False,True
435,-1.256367,-0.696614,0.293982,0.632486,0.556505,False,True,False,False,True,...,True,False,False,True,False,False,True,False,True,False


In [209]:
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ],
 "max_depth"        : [3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [1, 3, 5, 7],
 "gamma"            : [0.0, 0.1, 0.2, 0.3, 0.4],
 "colsample_bytree" : [0.3, 0.4, 0.5, 0.7]
    
}
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()

random_search = RandomizedSearchCV(classifier, param_distributions=params, n_iter=5, scoring='roc_auc', n_jobs=-1,cv=5,verbose=3)

random_search.fit(X_train,y_train)
print(f'{random_search.best_estimator_}')

Fitting 5 folds for each of 5 candidates, totalling 25 fits
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.4, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)


In [210]:
classifier = xgb.XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.4, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.25, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=7,  monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None)

from sklearn.model_selection import cross_val_score
score = cross_val_score(classifier, X, y, cv=10)
score.mean()
classifier.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.4, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.25, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=7, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [211]:
predictions = classifier.predict(X_test)

In [212]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


# TESTING PART
# IMPORT CSV
test = pd.read_csv('test_heart.csv', sep=',')
#FILTER VALUES
# chol_median = test.loc[test['Cholesterol'] != 0, 'Cholesterol'].median()
# test['Cholesterol'].replace(0, chol_median, inplace=True)

# resting_median = test.loc[test['RestingBP'] != 0, 'RestingBP'].median()
# test['RestingBP'].replace(0, resting_median, inplace=True)

temp_df = pd.get_dummies(test, columns=features_to_be_scaled)
temp_df[['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']] = std.transform(temp_df[['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']])
X_new = temp_df.drop(['id'], axis=1)



# # Calculating the accuracy
print(X_new)
# # PREDICT AND PUT INTO PANDAS DATAFRAME
predictions = classifier.predict(X_new)
id_to_prediction_df = pd.DataFrame({
    'id': test['id'],
    'HeartDisease': predictions
})

# # OUTPUT VIA CSV\a
file_name = './submissiontest.csv'
id_to_prediction_df.to_csv(file_name, index=False)

Accuracy: 85.57%
          Age  RestingBP  Cholesterol     MaxHR   Oldpeak  Sex_F  Sex_M  \
0   -1.149051  -0.976620     0.989774  1.785039  0.270302  False   True   
1    0.353374  -0.248604    -1.821229  0.473513  0.079501  False   True   
2    1.319219  -0.696614     0.980497  0.592743 -0.492905  False   True   
3    2.070431   1.543437    -1.821229 -0.599553 -0.874508   True  False   
4   -0.827103  -1.256627     0.386754  0.155568  0.079501  False   True   
..        ...        ...          ...       ...       ...    ...    ...   
271  0.246058   0.983424     0.312536 -0.480323  0.556505  False   True   
272 -0.934419  -0.024599     0.934111  0.314541 -0.874508   True  False   
273 -0.075890  -0.360606    -1.821229 -1.195701 -0.874508  False   True   
274 -2.007579  -0.584611    -0.039999  1.506836 -0.874508  False   True   
275 -0.183206   0.423411     1.926775 -0.480323  1.033509  False   True   

     ChestPainType_ASY  ChestPainType_ATA  ChestPainType_NAP  ...  \
0            